# 五、生成

本章介绍 RAG 系统中**生成阶段**的关键优化技术，包括后处理（信息压缩、重排序）和参考引用。

## 5.1 后处理

后处理是 RAG 系统中的一个关键环节，通常发生在检索阶段之后，旨在优化和提升检索和生成结果的质量和相关性。

<img src="./postprocess.png" width="680px">

- Embedding Model : 将文档段落编码为向量的嵌入模型。
- Retriever：通过嵌入模型编码用户问题 query，并返回嵌入问题附近的任意编码文档 documents。
- 后处理（可选）: Compressor 提取关键信息；ReRanker 根据某规则重新计算 query 与 documents 间的得分。
- Language Model : 语言模型用于接收来自检索器或重排器的记录以及问题，并返回答案。


**为什么进行后处理？**

1. **优化信息密度**：由于 LLM 对输入的字数有限制，后处理可以通过筛选和压缩信息从大量检索结果中提炼出最核心的信息。这样可以确保即使在字数限制内，也能向模型提供最相关和最有价值的数据，从而生成高质量和高密度的信息内容。

1. **提高生成效率**：在有限的字数下，模型需要处理的信息越精炼，其生成答案的效率就越高。后处理通过去除冗余和不相关信息，确保模型专注于最关键的内容，这样可以在有限的交互中快速生成准确和有用的回答。

1. **确保内容的连贯性**：在字数受限的情况下，模型可能无法一次性接收并处理所有相关信息。后处理可以帮助组织和结构化信息，使其在生成阶段能够以连贯的方式呈现，避免因字数限制而导致的信息断层或不完整。

1. **提升生成文本的质量**：后处理不仅可以优化输入数据，还可以通过调整生成策略来提升输出文本的质量。例如，通过设置优先级，确保最重要的信息首先被包含在生成的文本中，即使在字数限制的情况下也能保证回答的核心价值。


后处理通常可包括以下方面：

1. **信息压缩**：由于检索阶段可能会返回大量相关信息，为了提高生成阶段的效率和性能，可对大量信息进行压缩，提取最关键的内容。这可能涉及到提取关键句子、短语或概念，以便在生成答案时能够集中于最相关的信息。

1. **重新排序**：根据与用户查询的相关性对检索结果进行重新排序。这通常基于文档与查询的匹配程度、文档的权威性、用户的历史偏好等因素。通过这种方式，最相关和最有用的信息会被放在最前面，以供生成模型优先考虑。

通过这些后检索处理步骤，RAG 系统能够确保生成阶段的输入是精炼、相关且高质量的，从而提高最终生成文本的准确性和用户满意度。

### 5.1.1 信息压缩

**为什么进行信息压缩？**

信息压缩通常指的是减少数据量而不丢失关键信息的过程。以往的方法在整合 LLMs 到检索式问答框架中存在一定的局限性，如计算成本高、对长文本的处理不足等。为了提高生成阶段的效率和性能，可对检索阶段返回的大量信息进行压缩，提取最关键的内容，以便在生成答案时能够集中于最相关的信息。

**信息压缩**

信息压缩可以对于 retriever 返回的内容进行压缩即上下文压缩（contextual compression）；也可以对于已组成的结构化 prompt 信息进行压缩即提示压缩（prompt compression）。


1）**上下文压缩（Contextual Compression）案例**
- 上下文压缩：使用给定查询的上下文对文档进行压缩，从而只返回相关信息，而不是立即按原样返回检索到的文档。这里的 "压缩 "既指压缩单个文档的内容，也指整体过滤掉文档。可以借助 langchain 包中的 [ContextualCompressionRetriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.contextual_compression.ContextualCompressionRetriever.html) 实现此功能。

**1. 原始检索效果展示**

为了演示 RAG 系统中上下文压缩的必要性，我们使用南瓜书《机器学习公式详解》作为示例。这本书是《西瓜书》（周志华教授的《机器学习》）的配套讲解，涵盖了机器学习中各类算法的数学公式推导和详解。

我们将文档分割成多个文档块，然后进行检索。通过对比压缩前后的检索结果，展示上下文压缩如何过滤掉不相关的文档块，提高检索质量。

> **数据说明**：本教程使用与「4. 检索阶段」相同的数据集，保持教程连贯性。


#### 环境准备

在开始之前，请确保安装以下依赖：

```bash
pip install langchain langchain-community langchain-core
pip install chromadb pymupdf
pip install sentence-transformers  # 用于重排序模型
pip install llmlingua   # 用于提示压缩
pip install modelscope  # 用于国内下载模型

```


In [ ]:
# 导入必要的库
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import re
import json
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain_community.chat_models import ChatZhipuAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever
from dotenv import load_dotenv
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# 加载环境变量
load_dotenv()

# 检查 API Key 是否已设置
api_key = os.environ.get("ZHIPUAI_API_KEY")
llm = ChatZhipuAI(
    model="glm-4-flash", 
    temperature=0.0, 
    timeout=1200, 
    max_retries=3, 
    api_key=api_key
)

print("✅ 环境准备完成")

✅ 环境准备完成


In [ ]:
# ==================== 数据准备（与检索阶段保持一致）====================
# 使用与 "4. 检索阶段" 相同的南瓜书数据集，确保教程连贯性

# 数据路径（相对于当前notebook所在目录）
pdf_path = "../3. 索引阶段/data/pumpkin_book.pdf"
qa_path = '../3. 索引阶段/data/train_dataset.json'

# 初始化 Embedding 模型
embedding = ZhipuAIEmbeddings(
    api_key=os.environ.get('ZHIPUAI_API_KEY'),
    model="embedding-3"
)
print("✅ Embedding模型初始化成功")

# 加载问答数据集
with open(qa_path, 'r', encoding='utf-8') as file:
    qa_pairs = json.load(file)
print(f"✅ 加载了 {len(qa_pairs)} 个问答对")

# ---------- 向量库构建函数（与检索阶段保持一致）----------
def clean_text(text: str):
    """文本清理函数"""
    text = re.sub(r'→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》\n←_←', '', text)
    text = re.sub(r'→_→\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n←_←', '', text)
    text = re.sub(r'\s+', '', text)
    text = re.sub(r'\n+', '', text)
    return text

def build_vectorstore(pdf_path, embedding, chunk_size=2000, chunk_overlap=100, persist_directory="./chroma_db"):
    """构建或加载向量数据库"""
    if os.path.exists(persist_directory) and os.listdir(persist_directory):
        print(f"发现已存在的向量数据库: {persist_directory}，正在加载...")
        try:
            vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)
            count = vectorstore._collection.count() 
            print(f"✅ 加载成功！库中包含 {count} 个文档块。")
            return vectorstore
        except Exception as e:
            print(f"⚠️ 加载失败 ({e})，将重新构建...")
    
    print("未发现现有向量库或加载失败，开始重新构建...")
    loader = PyMuPDFLoader(pdf_path)
    pdf_pages = loader.load()
    data_pages = pdf_pages[13:-13]  # 去掉前后的目录和附录页
    for page in data_pages:
        page.page_content = clean_text(page.page_content)
    
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, separator='')
    split_docs = text_splitter.split_documents(data_pages)
    for split_doc in tqdm(split_docs, desc="处理文档"):
        split_doc.metadata['chunk_size'] = chunk_size
        split_doc.metadata['chunk_overlap'] = chunk_overlap
    
    batch_size = 16
    vectorstore = None
    print(f"共{len(split_docs)}个文档块，将分{(len(split_docs) + batch_size - 1) // batch_size}批处理...")
    
    for i in tqdm(range(0, len(split_docs), batch_size), desc="批量构建向量库"):
        batch = split_docs[i:i+batch_size]
        if vectorstore is None:
            vectorstore = Chroma.from_documents(documents=batch, embedding=embedding, persist_directory=persist_directory)
        else:
            vectorstore.add_documents(documents=batch)
    
    print(f"✅ 向量数据库构建完成并保存至 {persist_directory}！")
    return vectorstore

# 构建向量库
print("正在初始化向量数据库...")
vectorstore = build_vectorstore(pdf_path, embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# ---------- 辅助函数 ----------
def print_docs(docs, title="检索结果"):
    """打印检索结果"""
    print(f"{'='*20} {title} {'='*20}")
    for i, doc in enumerate(docs, 1):
        page = doc.metadata.get('page', 'N/A')
        print(f"Document {i} [页码:{page}]:")
        print(doc.page_content[:150] + "..." if len(doc.page_content) > 150 else doc.page_content)
        print()

# 执行原始检索
query = "在决策树基于“密度”属性进行划分时，南瓜书计算的候选划分点集合 Ta 包含了哪些具体数值？"
original_docs = retriever.invoke(query)

print_docs(original_docs, "原始检索结果 (Top 5)")

# 计算原始上下文总长度
original_length = sum(len(doc.page_content) for doc in original_docs)
print(f">>> 原始上下文总长度: {original_length} 字符")

✅ Embedding模型初始化成功
✅ 加载了 119 个问答对
正在初始化向量数据库...
发现已存在的向量数据库: ./chroma_db，正在加载...
✅ 加载成功！库中包含 170 个文档块。
==================== 原始检索结果 (Top 5) ====================
Document 1 [页码:51]:
式(4.7)可知，此时i依次取1到16，那么“密度”这个属性的候选划分点集合为Ta={0.243+0.2452,0.245+0.3432,0.343+0.3602,0.360+0.4032,0.403+0.4372,0.437+0.4812,0.481+0.5562,0.556+0.5932,0.5...

Document 2 [页码:49]:
Gini_index(D,敲声=浊响)=0.450Gini_index(D,敲声=沉闷)=0.494Gini_index(D,敲声=清脆)=0.439Gini_index(D,纹理=清晰)=0.286Gini_index(D,纹理=稍稀)=0.437Gini_index(D,纹理=模糊)=0.403...

Document 3 [页码:50]:
(2)遍历所有属性，找到最优划分属性a∗，然后根据a∗的最优划分点v∗将特征空间划分为两个子空间，接着对每个子空间重复上述步骤，直至满足停止条件.这样就生成了一棵CART回归树，假设最终将特征空间划分为M个子空间R1,R2,···,RM，那么CART回归树的模型式可以表示为f(x)=MXm=1cmI...

Document 4 [页码:52]:
经过四次划分已无空白部分，表示决策树生成完毕，从图4-2(d)中可以清晰地看出好瓜与坏瓜的分类边界。含糖率密度0.60.40.20.20.40.60.80(a)第一次划分含糖率密度0.60.40.20.20.40.60.80(b)第二次划分含糖率密度0.60.40.20.20.40.60.80(c)...

Document 5 [页码:48]:
易证以上两式之和等于1，证明过程如下|Y|=3Xk=1p2k+|Y|=3Xk=1Xk′̸=kpkpk′=(p1p1+p2p2+p3p3)+(p1p2+p1p3+p2p1+p2p3+p3p1+p3p2)=(p1p1+p1p2+p1p3)+(p

**2. 实施上下文压缩**

从上面的原始检索结果可以看到，返回了 5 个文档块，但并非所有文档块都与查询高度相关。有些文档块可能只是提及了某些相关关键词，但并不能真正回答问题。

如果不进行压缩，这些噪音会：

1.  **浪费 Token**：不仅增加了成本，还占用了宝贵的上下文窗口。
2.  **干扰回答**：过多不相关的信息可能导致模型生成偏离主题的回答。

现在我们应用 `ContextualCompressionRetriever` 来过滤掉相关性低的文档块。


In [38]:
# 1. 定义过滤器
# similarity_threshold 是核心参数，设定为 0.6 意味着过滤掉相关性低于 0.6 的文档
embeddings_filter = EmbeddingsFilter(embeddings=embedding, similarity_threshold=0.6)

# 2. 构建压缩检索器
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,  # 使用 EmbeddingsFilter 作为压缩器
    base_retriever=retriever  # 使用原始检索器
)

# 3. 执行压缩检索
print(f"Query: {query}")
compressed_docs = compression_retriever.invoke(query)
print_docs(compressed_docs, "压缩后检索结果")

# 计算压缩后上下文总长度
compressed_length = sum(len(doc.page_content) for doc in compressed_docs)
print(f"\n>>> 压缩后上下文总长度: {compressed_length} 字符")
print(f">>> 压缩率: {(1 - compressed_length/original_length)*100:.1f}%")

Query: 在决策树基于“密度”属性进行划分时，南瓜书计算的候选划分点集合 Ta 包含了哪些具体数值？
==================== 压缩后检索结果 ====================
Document 1 [页码:51]:
式(4.7)可知，此时i依次取1到16，那么“密度”这个属性的候选划分点集合为Ta={0.243+0.2452,0.245+0.3432,0.343+0.3602,0.360+0.4032,0.403+0.4372,0.437+0.4812,0.481+0.5562,0.556+0.5932,0.5...


>>> 压缩后上下文总长度: 1454 字符
>>> 压缩率: 73.4%


可以看出压缩后，检索器只返回了与查询高度相关的文档块，过滤掉了相关性较低的内容。这不仅减少了 Token 消耗，还提高了后续生成的质量。

**3. 生成效果对比**

接下来，我们对比使用原始上下文和压缩后上下文进行生成的效果差异。

In [39]:
# 定义生成链
template = """仅根据以下背景回答问题。如果背景中没有答案，请回答不知道。
背景信息：
{context}

问题: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(f"Query: {query}")

# 1. 使用原始文档生成
print("\n" + "=" * 20 + " 基于 [原始] 上下文的回答 " + "=" * 20)
try:
    original_response = chain.invoke({"context": format_docs(original_docs), "question": query})
    print(original_response)
except Exception as e:
    print(f"生成失败: {e}")

# 2. 使用压缩后的文档生成
print("\n" + "=" * 20 + " 基于 [压缩后] 上下文的回答 " + "=" * 20)
try:
    compressed_response = chain.invoke({"context": format_docs(compressed_docs), "question": query})
    print(compressed_response)
except Exception as e:
    print(f"生成失败: {e}")

Query: 在决策树基于“密度”属性进行划分时，南瓜书计算的候选划分点集合 Ta 包含了哪些具体数值？

==================== 基于 [原始] 上下文的回答 ====================
在决策树基于“密度”属性进行划分时，南瓜书计算的候选划分点集合 Ta 包含以下具体数值：

0.243 + 0.2452
0.245 + 0.3432
0.343 + 0.3602
0.360 + 0.4032
0.403 + 0.4372
0.437 + 0.4812
0.481 + 0.5562
0.556 + 0.5932
0.593 + 0.6082
0.608 + 0.6342
0.634 + 0.6392
0.639 + 0.6572
0.657 + 0.6662
0.666 + 0.6972
0.697 + 0.7192
0.719 + 0.7742

==================== 基于 [压缩后] 上下文的回答 ====================
南瓜书计算的候选划分点集合 Ta 包含了以下具体数值：
0.243+0.2452, 0.245+0.3432, 0.343+0.3602, 0.360+0.4032, 0.403+0.4372, 0.437+0.4812, 0.481+0.5562, 0.556+0.5932, 0.593+0.6082, 0.608+0.6342, 0.634+0.6392, 0.639+0.6572, 0.657+0.6662, 0.666+0.6972, 0.697+0.7192, 0.719+0.7742。


通过上下文压缩，我们显著减少了输入 Token 数量，同时大模型依然能够生成准确的回答。这在实际应用中可以有效降低 API 调用成本并提升响应速度。

2）**提示压缩（Prompt Compression）案例**


- 提示压缩：一种从粗到细的提示语压缩策略，作为 Token 预算控制器，它能在高压缩率下保持语义完整性。这里我们使用 [LLMLingua](https://github.com/microsoft/LLMLingua) 工具包，这是一种轻量且强大的提示压缩方法。它利用从 GPT-4 蒸馏的数据训练 BERT 级编码器进行 token 分类，在任务无关的压缩场景中表现出色，能有效识别并剔除 Prompt 中的冗余 token。


In [50]:
# 我们先使用 ModelScope 下载模型需要用到的模型
llmlingua_model_dir = snapshot_download('microsoft/llmlingua-2-xlm-roberta-large-meetingbank')

In [59]:
import json
import os
from llmlingua import PromptCompressor
from modelscope import snapshot_download

# 1. 使用检索到的南瓜书文档作为上下文
# question = "什么是决策树？它的基本原理是什么？"
question = "在决策树基于“密度”属性进行划分时，南瓜书计算的候选划分点集合 Ta 包含了哪些具体数值？"

# 使用之前检索到的压缩后文档作为上下文
context = [doc.page_content for doc in original_docs]

# 2. 测试原始 Prompt 效果
original_prompt = "\n".join(context) + "\n" + question
print(f"原始 Prompt 长度: {len(original_prompt)} 字符")
response_original = llm.invoke(original_prompt)
print(f"\n>>> 原始回答:\n{response_original.content}")

# 3. 初始化压缩模型

# 初始化压缩器
llm_lingua = PromptCompressor(
    model_name=llmlingua_model_dir,
    use_llmlingua2=True,
    device_map="cpu"
)

# 4. 执行压缩
compressed_result = llm_lingua.compress_prompt(
    context,
    instruction="",
    question=question,
    rate=0.7,  # 压缩到原始的 30%
    force_tokens=["\n", "。", "，", "："]
)
compressed_prompt_str = compressed_result['compressed_prompt']
final_prompt = f"{compressed_prompt_str}\n\n基于以上信息，回答问题：{question}"

print(f"压缩后 Prompt 长度: {len(final_prompt)} 字符")

response_compressed = llm.invoke(final_prompt)
print(f"\n>>> 压缩后回答:\n{response_compressed.content}")


原始 Prompt 长度: 5511 字符

>>> 原始回答:
根据您提供的文本内容，式(4.7)用于计算连续属性“密度”的候选划分点集合 Ta。该集合是通过取每两个相邻取值的中点来生成的。以下是根据文本内容计算出的 Ta 集合中的具体数值：

1. 0.243 + 0.245 / 2 = 0.244
2. 0.245 + 0.343 / 2 = 0.279
3. 0.343 + 0.360 / 2 = 0.357
4. 0.360 + 0.403 / 2 = 0.381
5. 0.403 + 0.437 / 2 = 0.415
6. 0.437 + 0.481 / 2 = 0.454
7. 0.481 + 0.556 / 2 = 0.518
8. 0.556 + 0.593 / 2 = 0.574
9. 0.593 + 0.608 / 2 = 0.601
10. 0.608 + 0.634 / 2 = 0.621
11. 0.634 + 0.639 / 2 = 0.636
12. 0.639 + 0.657 / 2 = 0.647
13. 0.657 + 0.666 / 2 = 0.663
14. 0.666 + 0.697 / 2 = 0.672
15. 0.697 + 0.719 / 2 = 0.703
16. 0.719 + 0.774 / 2 = 0.748

因此，候选划分点集合 Ta 包含以下具体数值：

{0.244, 0.279, 0.357, 0.381, 0.415, 0.454, 0.518, 0.574, 0.601, 0.621, 0.636, 0.647, 0.663, 0.672, 0.703, 0.748}


Token indices sequence length is longer than the specified maximum sequence length for this model (1057 > 512). Running this sequence through the model will result in indexing errors


压缩后 Prompt 长度: 3727 字符

>>> 压缩后回答:
根据您提供的信息，在决策树基于“密度”属性进行划分时，候选划分点集合 \( T_a \) 包含了以下具体数值：

{0.243, 0.245, 0.343, 0.360, 0.403, 0.437, 0.481, 0.556, 0.593, 0.608, 0.634, 0.657, 0.666, 0.697, 0.719, 0.774}

这些数值是“密度”属性在数据集中已观测到的可能取值。在计算候选划分点时，通常会使用这些数值的中点作为候选划分点，但根据您提供的信息，这里直接列出了所有已观测到的取值。


`compress_prompt` 返回的信息中包含了压缩后的 prompt 以及压缩的统计信息。可以看出它节省了大量的 Token，同时保持了语义的完整性。

In [54]:
print(json.dumps(compressed_result, indent=2, ensure_ascii=False))

{
  "compressed_prompt": "，，={0.243+0.2452,0.245+0.3432,0.343+0.3602,0.360+0.4032,0.403+0.4372,0.437+0.4812,0.481+0.5562,0.556+0.5932,0.593+0.6082,0.608+0.6342,0.634+0.6392,0.639+0.6572,0.657+0.6662,0.666+0.6972,0.697+0.7192,0.719+0.7742}4.4.2式(4(4.2，， λ∈{−,+，=−时有Dλt=Datt，=+时有Dλt=Da>tt。 4.4.3式(4(4.2，(4.2|Dv||改为式(4.11)的 ̃rv，(4.1.10)的 ̃pk(4.9)的ρ。(4.9)(4.10)(4.11)中的权重wx， 初始化为1。，， 除编号为8、10的两个样本在此属性缺失之外，， 而编号为8、10的两个样本则要按比例同时划入三个子集。， 稍糊子集包含样本7、9、13、14、17共5个样本，，，， 而此时各样本的权重wx初始化为1， 因此编号为8、10的两个样本分到稍糊、清晰、模糊三个子集的权重分别为515,715和315。 4.5多变量决策树本节内容也通俗易懂，。 4.5.1图(4， 离散属性不可以重复使用，。 4.5.2图(4.11)的解释对照“西瓜书”中图4.10的决策树， 下面给出图4.11中的划分边界产出过程。 在下图4-2中， 斜纹阴影部分表示已确定标记为坏瓜的样本， 点状阴影部分表示已确定标记为好瓜的样本， 空白部分表示需要进一步划分的样本。 第一次划分条件是“含糖率0.126?”，(a，(a。(a)空白部分继续进行划分， 第二次划分条件是“密度0.381?”，(b)新增斜纹阴影部分所示，(b。(b)空白部分继续进行划分， 第三次划分条件是“含糖率0.205?”， 不满足此条件的样本直接标记为好瓜(c，(c。 在第三次划分的基础上对图4-2(c)空白部分继续进行划分， 第四次划分的条件是“密度0.560?”， 满足此条件的样本直接标记为好瓜(d)新增点状阴影部分所示， 而不满足此条件的样本直接标记为坏瓜(d。\n\n_index(D,敲声=浊响)=0(D,敲声=沉闷)=0=清脆)=0(D,纹理=清晰),纹理=稍稀)=0=模糊)=0.403

In [52]:

print(f"\n>>> 压缩统计:")
print(f"原始 Token 数: {compressed_result['origin_tokens']}")
print(f"压缩后 Token 数: {compressed_result['compressed_tokens']}")
# 计算压缩率（rate 可能是字符串或浮点数）
rate = compressed_result.get('rate', 0)
print(f"压缩率: {rate}")


>>> 压缩统计:
原始 Token 数: 5263
压缩后 Token 数: 2375
压缩率: 45.1%


通过提示压缩，我们将上下文和用户查询整合为 prompt 后，显著减少了整体字符数，从而降低了调用大模型接口的成本。且压缩后的 prompt 仍然能保持较好的语义完整性，大模型依然能够生成准确的回答。

### 5.1.2 重排序 (Reranking)

检索问答（RAG）系统中，**重排序（Reranking）** 是提升检索准确率的关键步骤，通常位于“两阶段检索”流程的第二阶段。

#### 1. 为什么要引入重排序？

<img src="reranker2.png" width="680px">

在传统的向量检索（Vector Search）中，我们主要依赖 Embedding 模型计算查询与文档的余弦相似度。虽然这种方法速度快，但存在局限性：
- **语义压缩损失**：Embedding 将丰富的文本信息压缩为固定维度的向量，不可避免地会丢失细节。
- **缺乏交互与上下文**：向量检索通常基于预先计算好的文档嵌入（Index），这些嵌入在创建时并未考虑特定查询的上下文。这种“双编码器”模式下，查询和文档是独立编码的，无法捕捉它们之间细微的词汇或逻辑交互。

因此，仅靠向量检索往往难以保证送入的文档都是最相关的，这就需要重排序来进行“精修”。

#### 2. 工作原理：两阶段检索与交叉编码器

为了兼顾**速度**和**精度**，工业界通常采用“漏斗”式的两阶段检索架构：

1.  **检索（Retrieval）**：使用向量检索或关键词检索（BM25），从海量文档中快速筛选出 Top-N（如前 100 个）候选文档。这一步注重**召回率**，目的是“不遗漏”。
2.  **重排（Reranking）**：使用**重排序模型**对这 Top-N 个文档进行逐一打分，作为**数据过滤**机制，精确计算它们与查询的相关性，并重新排序，最终只保留 Top-K（如前 5 个）给大模型。这一步注重**准确率**，目的是“去伪存真”。

**为什么重排序更准？**

重排序模型通常基于**交叉编码器（Cross-Encoder） **实现。不同于向量检索中查询与文档独立编码的**双编码器（Bi-Encoder）**，交叉编码器将查询和文档文本拼接后一起输入语言模型（如 BERT），通过深层的注意力机制直接计算两者的相关性得分（通常为 0~1 之间的概率值）。

- **优势**：能精准捕捉语义细节（如否定、因果关系），准确率远高于向量检索。
- **劣势**：无法预先构建文档索引，必须在查询时实时计算，计算开销大、速度慢，因此仅适用于对少量候选文档进行精细排序。

#### 3. 模型选择与效果
引入重排序可以显著提升 RAG 系统的最终效果。如下图所示，使用 `bge-reranker` 或 `cohere-reranker` 后，检索准确率明显优于未重排序（WithoutReranker）的情况 *「以下内容为在私有数据集上的效果，不同数据集效果可能不同」*。

<img src="reranker_perform.png" width="680px">

为客观评估与选择模型，建议参考 [MTEB/CMTEB](https://huggingface.co/spaces/mteb/leaderboard)（MTEB 是多任务基准测试集，CMTEB 是其中文版本） 等公共权威榜单。这些榜单提供了不同模型在标准化任务上的性能比较，可作为选型的重要依据。

目前常见的重排序方案主要包括：
- **开源模型**：[BAAI/bge-reranker](https://huggingface.co/BAAI/bge-reranker-base) 系列（bge-reranker-base/base/large），支持本地私有化部署，效果在开源界领先。
- **商业 API**：[Cohere Rerank](https://docs.cohere.com/docs/rerank-2) 和 [Jina Reranker](https://jina.ai/reranker/)，提供高性能的 API 服务，无需自行维护基础设施。

接下来，我们将演示如何使用 `bge-reranker-large` 模型在本地进行重排序。

In [ ]:
# 这里我们用 modelscope 下载模型
model_dir = snapshot_download('BAAI/bge-reranker-large') # 也可以下载 base 「bge-reranker-base」模型，更轻量级一些，当时效果稍差

我们使用 `LangChain` 提供的抽象组件来实现重排序。这在构建 RAG 管道时会更加方便，因为可以无缝集成到 `ContextualCompressionRetriever` 中。

由于 LangChain 的 `HuggingFaceCrossEncoder` 封装有时需要配合特定的依赖，或者为了更灵活的控制，我们可以自定义一个简单的 CrossEncoder 类，或者直接使用 `HuggingFaceCrossEncoder`（如果环境中安装了 `langchain_community`）。

以下是基于 LangChain 的实现示例：

In [61]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# 1. 初始化 CrossEncoder 重排序模型
rerank_model = HuggingFaceCrossEncoder(model_name=model_dir)

# 2. 创建重排序器 (Compressor)
# top_n 控制最终保留最相关的多少个文档
reranker = CrossEncoderReranker(model=rerank_model, top_n=3)

# 3. 使用之前构建的 retriever 进行检索，然后重排序
# 构建带重排序的压缩检索器
rerank_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=retriever  # 使用之前构建的南瓜书文档检索器
)

# 4. 执行检索+重排序
rerank_query = "南瓜书中对模型评估中的“代价矩阵”进行了怎样的公式变形？请结合 cost01 和 cost10 说明。"

print(f"Query: {rerank_query}")
print("\n" + "=" * 20 + " 原始检索结果 (Top 5) " + "=" * 20)
original_results = retriever.invoke(rerank_query)
for i, doc in enumerate(original_results, 1):
    print(f"\n[{i}] {doc.page_content[:300]}...")

print("\n" + "=" * 20 + " 重排序后结果 (Top 3) " + "=" * 20)
reranked_results = rerank_retriever.invoke(rerank_query)
for i, doc in enumerate(reranked_results, 1):
    print(f"\n[{i}] {doc.page_content[:300]}...")

Query: 南瓜书中对模型评估中的“代价矩阵”进行了怎样的公式变形？请结合 cost01 和 cost10 说明。

==================== 原始检索结果 (Top 5) ====================

[1] 中将cost01记为cost+−，cost10记为cost−+。本公式还可以作如下恒等变形E(f;D;cost)=1mm+×1m+Xxi∈D+I(f(xi̸=yi))×cost+−+m−×1m−Xxi∈D−I(f(xi̸=yi))×cost−+=m+m×1m+Xxi∈D+I(f(xi̸=yi))×cost+−+m−m×1m−Xxi∈D−I(f(xi̸=yi))×cost−+其中m+和m−分别表示正例子集D+和反例子集D−的样本个数。1m+Pxi∈D+I(f(xi̸=yi))表示正例子集D+预测错误样本所占比例，即假反例率FNR。1m−Pxi∈D−I(f(xi̸=yi))表示反例子集D−...

[2] 由书中上下文可知，式(10.28)是如下优化问题的解。minw1,w2,...,wmmXi=1xi−Xj∈Qiwijxj22s.t.Xj∈Qiwij=1若令xi∈Rd×1,Qi={q1i,q2i,...,qni}，则上述优化问题的目标函数可以进行如下恒等变形mXi=1xi−Xj∈Qiwijxj22=mXi=1Xj∈Qiwijxi−Xj∈Qiwijxj22=mXi=1Xj∈Qiwij(xi−xj)22=mXi=1∥Xiwi∥22=mXi=1wiTXTiXiwi其中wi=(wiq1i,wiq2i,...,wiqni)∈Rn×1，Xi= xi−xq1i,xi−xq2i,...,xi−xqni∈Rd...

[3] 所以ˆµ0=1m0K10=12κ(x1,x1)+κ(x1,x3)κ(x2,x1)+κ(x2,x3)κ(x3,x1)+κ(x3,x3)κ(x4,x1)+κ(x4,x3)∈R4×1ˆµ1=1m1K11=12κ(x1,x2)+κ(x1,x4)κ(x2,x2)+κ(x2,x4)κ(x3,x2)+κ(x3,x4)κ(x4,x2)+κ(x4,x4)∈R4×1根据此结果易得ˆµ0,ˆµ1的一般形式为ˆµ0=1m0K10=1m0Px∈X0κ(x1,x)Px∈X0κ(x2

可以看到，重排序后果与原始检索结果的排序有所不同。Cross-Encoder 模型能够更精确地计算查询与文档的相关性，从而将最相关的文档排在前面。检索回来的都是公式内容，而经过 reranker 后，将 cost 相关的公式内容排在前面。

有了重排序，我们可以扩大检索范围，尽可能召回相关文档，同时保证送给模型的内容质量，以确保生成效果。

## 5.2 参考引用

**为什么要采用参考引用？**
参考引用指示生成的内容来自于检索的哪些文档。

1. **提供可解释性**：通过参考引用，可以明确知道生成的答案中的不同部分来自于哪些源文档。这提供了一种可解释性，让用户或开发者能够追踪答案的来源，并评估答案的可靠性。

1. **增强答案的准确性**：参考引用可以帮助生成模型更准确地生成答案。通过引用相关文档中的内容，模型可以更好地利用检索到的知识来回答问题，减少生成不相关或错误信息的可能性。

1. **促进知识的整合**：参考引用允许模型将来自不同文档的信息进行整合，生成更全面和连贯的答案。通过引用多个相关文档，模型可以综合不同来源的知识，生成更加完善的答案。

1. **便于答案的后处理**：参考引用提供了一种结构化的方式来表示答案中的知识来源。这种结构化信息可以方便地进行后处理，如将文档 ID 替换为实际的文档标题、链接或摘要等，使答案更加人性化和易于理解。

1. **支持答案的评估和改进**：通过参考引用，可以方便地评估生成答案的质量。可以比较生成的答案与引用文档的相关性和一致性，从而识别模型生成答案的优缺点。这有助于发现模型的局限性，并为进一步改进模型提供指导。

下面将通过案例帮助大家理解 _参考引用_ 及其实现，面对多个文档，如何分 chunks 方便索引。


接下来我们演示如何使用 LangChain 实现参考引用功能。我们将使用南瓜书（机器学习公式详解）作为知识库，让模型在回答问题时标注信息来源（包括页码）。

这里我们用两种方式实现，第一种是直接使用 `Prompt` 的方式，第二种是使用 `Pydantic` 进行结构化输出。
首先演示第一种方式，这里通过指令模板引导模型在回答中直接标注引用，实现简单直观。输出为纯文本，易于解析和展示。

In [ ]:
# 利用之前构建的南瓜书向量存储和检索器

# 定义带引用的生成模板
citation_template = """基于以下背景信息回答问题。请在回答中标注信息来源，格式为 [来源X]。

背景信息：
{context}

问题：{question}

请按以下格式回答：
1. 先给出答案，在相关陈述后标注 [来源1]、[来源2] 等
2. 最后列出所有引用的来源
"""

citation_prompt = ChatPromptTemplate.from_template(citation_template)

# 检索相关文档
# citation_query = "什么是支持向量机(SVM)？它的核心思想是什么？"
citation_query = "南瓜书中对支持向量机（SVM）的对率损失函数式(6.33)是如何推导的？"
citation_docs = retriever.invoke(citation_query)

print(f"Query: {citation_query}")
print("\n" + "=" * 20 + " 检索到的文档 " + "=" * 20)
for i, doc in enumerate(citation_docs, 1):
    print(f"\n[来源{i}] {doc.page_content[:300]}...")

Query: 南瓜书中对支持向量机（SVM）的对率损失函数式(6.33)是如何推导的？

==================== 检索到的文档 ====================

[来源1] 例，则上式可改写为ℓ(w,b)=Pmi=1ln1+e−(wTxi+b),yi=+1Pmi=1ln1+ewTxi+b,yi=−1=mXi=1ln1+e−yi(wTxi+b)此时上式的求和项正是式(6.33)所表述的对率损失。6.4.6式(6.41)的解释参见式(6.13)的解释6.5...

[来源2] 第6章支持向量机在深度学习流行之前，支持向量机及其核方法一直是机器学习领域中的主流算法，尤其是核方法至今都仍有相关学者在持续研究。6.1间隔与支持向量6.1.1图6.1的解释回顾第5章5.2节的感知机模型可知，图6.1中的黑色直线均可作为感知机模型的解，因为感知机模型求解的是能将正负样本完全正确划分...

[来源3] 6.4.4式(6.40)将式(6.37)、式(6.38)和(6.39)代入式(6.36)可以得到式(6.35)的对偶问题，有12∥w∥2+CmXi=1ξi+mXi=1αi 1−ξi−yi wTxi+b−mXi=1µiξi=12∥w∥2+mXi=1αi 1−yi wTxi+b+CmXi=1ξi...

[来源4] (1)式(6.6)中的未知数是w和b，式(6.11)中的未知数是α，w的维度d对应样本特征个数，α的维度m对应训练样本个数，通常m≪d，所以求解式(6.11)更高效，反之求解式(6.6)更高效；(2)式(6.11)中有样本内积xTxji这一项，后续可以很自然地引入核函数，进而使得支持向量机也能对在原...

[来源5] 式(3.11)的推导思路如下：首先根据定理3.1推导出Eˆw是ˆw的凸函数，接着根据定理3.2推导出式(3.11)。下面按照此思路进行推导。由于式(3.10)已推导出Eˆw关于ˆw的一阶导数，接着基于此进一步推导出二阶导数，即Hessian矩阵。推导过程如下：∇2Eˆw=∂∂ˆwT∂Eˆw∂ˆw...


In [ ]:
def format_docs_with_sources(docs):
    """格式化文档，添加来源编号"""
    formatted = []
    for i, doc in enumerate(docs, 1):
        page = doc.metadata.get('page', 'N/A')
        formatted.append(f"[来源{i}] (第{page}页) {doc.page_content}")
    return "\n\n".join(formatted)

# 生成带引用的回答
context_with_sources = format_docs_with_sources(citation_docs)

# 构建生成链
citation_chain = citation_prompt | llm | StrOutputParser()

# 执行生成
print("\n" + "=" * 20 + " 带引用的回答 " + "=" * 20)
citation_response = citation_chain.invoke({
    "context": context_with_sources,
    "question": citation_query
})
print(citation_response)


==================== 带引用的回答 ====================
1. 南瓜书中对支持向量机（SVM）的对率损失函数式(6.33)的推导如下：首先，将式(6.37)、式(6.38)和(6.39)代入式(6.36)可以得到式(6.35)的对偶问题。然后，通过一系列代数变换和条件约束，最终得到式(6.33)。具体推导过程涉及对数几率回归与支持向量机的关系，以及对数几率损失函数的变形。这一推导过程在“西瓜书”的第6章中有所描述 [来源1]。

2. 引用来源：
   [来源1] (第67页) 例，则上式可改写为ℓ(w,b)=Pmi=1ln(1+e−(wTxi+b)),yi=+1Pmi=1ln(1+ewTxi+b),yi=−1=mXi=1ln(1+e−yi(wTxi+b))此时上式的求和项正是式(6.33)所表述的对率损失。6.4.6式(6.41)的解释参见式(6.13)的解释6.5支持向量回归6.5.1式(6.43)的解释相比于线性回归用一条线来拟合训练样本，支持向量回归而是采用一个以f(x)=wTx+b为中心，宽度为2ϵ的间隔带，来拟合训练样本。落在带子上的样本不计算损失（类比线性回归在线上的点预测误差为0），不在带子上的则以偏离带子的距离作为损失（类比线性回归的均方误差），然后以最小化损失的方式迫使间隔带从样本最密集的地方穿过，进而达到拟合训练样本的目的。因此支持向量回归的优化问题可以写为minw,b12∥w∥2+CmXi=1ℓϵ(f(xi)−yi)其中ℓϵ(z)为“ϵ不敏感损失函数”（类比线性回归的均方误差损失）ℓϵ(z)=(0,if|z|⩽ϵ|z|−ϵ,if|z|>ϵ12∥w∥2为L2正则项，此处引入正则项除了起正则化本身的作用外，也是为了和软间隔支持向量机的优化目标保持形式上的一致，这样就可以导出对偶问题引入核函数，C为用来调节损失权重的正则化常数。6.5.2式(6.45)的推导同软间隔支持向量机，引入松弛变量ξi，令ℓϵ(f(xi)−yi)=ξi显然ξi⩾0，并且当|f(xi)−yi|⩽ϵ时，ξi=0，当|f(xi)−yi|>ϵ时，ξi=|f(xi)−yi|−ϵ，所以|f(xi)−yi|−ϵ⩽ξi|f(xi)−yi|⩽ϵ+ξi−ϵ−ξi⩽f(xi)−yi⩽ϵ+ξi因此支持向量回归的优化问题可以化为minw,b,ξi12∥

第二种方式利用 `Pydantic` 定义结构化输出格式，确保返回结果具有固定的数据结构，以提供置信度等元数据信息，但是需要模型支持结构化输出功能
，对指令遵循能力要求较高。

In [70]:
# 结构化输出示例 - 使用 Pydantic 解析引用
from typing import List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# 1. 定义结构化输出 Schema
class Citation(BaseModel):
    """单个引用"""
    source_id: int = Field(description="来源编号")
    quote: str = Field(description="引用的原文片段")

class AnswerWithCitations(BaseModel):
    """带引用的回答"""
    answer: str = Field(description="回答内容")
    citations: List[Citation] = Field(description="引用列表")
    confidence: float = Field(description="置信度，0-1之间")

# 2. 创建 Pydantic 解析器
parser = PydanticOutputParser(pydantic_object=AnswerWithCitations)

# 3. 构建带格式指令的 Prompt
structured_template = """基于以下背景信息回答问题，并提供引用。

背景信息：
{context}

问题：{question}

{format_instructions}
"""

structured_prompt = ChatPromptTemplate.from_template(structured_template)

# 4. 执行结构化生成
citation_query = "南瓜书中对支持向量机（SVM）的对率损失函数式(6.33)是如何推导的？"
structured_docs = retriever.invoke(citation_query)

try:
    structured_chain = structured_prompt | llm | parser
    result = structured_chain.invoke({
        "context": format_docs_with_sources(structured_docs),
        "question": citation_query,
        "format_instructions": parser.get_format_instructions()
    })
    
    print(f"Query: {citation_query}")
    print("\n" + "=" * 20 + " 结构化输出 " + "=" * 20)
    print(f"\n回答: {result.answer}")
    print(f"\n置信度: {result.confidence}")
    print(f"\n引用:")
    for cite in result.citations:
        print(f"  [来源{cite.source_id}]: {cite.quote[:50]}...")
except Exception as e:
    print(f"结构化解析失败: {e}")
    print("注意：结构化输出对模型能力有一定要求，某些模型可能无法完美遵循格式指令。")

Query: 南瓜书中对支持向量机（SVM）的对率损失函数式(6.33)是如何推导的？

==================== 结构化输出 ====================

回答: 南瓜书中对支持向量机（SVM）的对率损失函数式(6.33)的推导过程如下：首先，考虑对数几率回归模型，其损失函数为对数损失函数。在支持向量机中，使用对率损失函数来代替对数几率回归中的0/1损失函数，从而得到对率回归模型。具体推导如下：

1. 对数几率回归模型的对数损失函数为：
   ℓ(β) = ∑_{i=1}^{m} ln(y_i p(y_i|x_i;β) + (1−y_i) p(0|x_i;β))

2. 其中，p(y_i|x_i;β) 表示在给定特征 x_i 和参数 β 的情况下，样本 y_i 出现的概率，p(0|x_i;β) 表示在给定特征 x_i 和参数 β 的情况下，样本 y_i 不出现的概率。

3. 将 p(y_i|x_i;β) 和 p(0|x_i;β) 分别表示为指数函数的形式：
   p(y_i|x_i;β) = e^(β^T x_i) / (1 + e^(β^T x_i))
   p(0|x_i;β) = 1 / (1 + e^(β^T x_i))

4. 将上述表达式代入对数损失函数中，得到：
   ℓ(β) = ∑_{i=1}^{m} ln(y_i e^(β^T x_i) + (1−y_i) / (1 + e^(β^T x_i)))

5. 对上述表达式进行化简，得到对率损失函数式(6.33)：
   ℓ(β) = ∑_{i=1}^{m} [y_i ln(y_i e^(β^T x_i) + (1−y_i) / (1 + e^(β^T x_i))) + (1−y_i) ln(1 / (1 + e^(β^T x_i)))]

6. 在支持向量机中，正例和反例分别用 y_i = +1 和 y_i = −1 表示，因此可以将上述表达式中的 y_i 替换为 +1 和 −1，得到支持向量机的对率损失函数式(6.33)：
   ℓ(β) = ∑_{i=1}^{m} [y_i ln(y_i e^(β^T x_i) + (1−y_i) / (1 + e^(β^T x_i))) + (1−y_i) ln(1 / (1 + e^(β^T x_i)))]

置信度: 0

In [68]:
for i, doc in enumerate(structured_docs, 1):
    print(f"\n[来源{i}] {doc.page_content[:3000]}...")


[来源1] 对于第一个条件，当超平面满足该条件时，根据超平面的性质(4)可知，若yi=+1的正样本被划分到正空间（当然也可以将其划分到负空间），yi=−1的负样本被划分到负空间，以下不等式成立(wTxi+b⩾0,yi=+1wTxi+b⩽0,yi=−1对于第二个条件，首先设离超平面最近的正样本为x+∗，离超平面最近的负样本为x−∗，由于这两样本是离超平面最近的点，所以其他样本到超平面的距离均大于等于它们，即|wTxi+b|∥w∥⩾|wTx+∗+b|∥w∥,yi=+1|wTxi+b|∥w∥⩾|wTx−∗+b|∥w∥,yi=−1结合第一个条件中推导出的不等式，可将上式中的绝对值符号去掉并推得(wTxi+b∥w∥⩾wTx+∗+b∥w∥,yi=+1wTxi+b∥w∥⩽wTx−∗+b∥w∥,yi=−1基于此再考虑第二个条件，“位于正负样本正中间”等价于要求超平面到x+∗和x−∗这两点的距离相等，即wTx+∗+b∥w∥=wTx−∗+b∥w∥综上，支持向量机所要求的超平面所需要满足的条件如下wTxi+b∥w∥⩾wTx+∗+b∥w∥,yi=+1wTxi+b∥w∥⩽wTx−∗+b∥w∥,yi=−1|wTx+∗+b|∥w∥=|wTx−∗+b|∥w∥但是根据超平面的性质(2)可知，当等倍缩放法向量w和位移项b时，超平面不变，且上式也恒成立，因此会导致所求的超平面的参数w和b有无穷多解。因此为了保证每个超平面的参数只有唯一解，不妨再额外施加一些约束，例如约束x+∗和x−∗代入进超平面方程后的绝对值为1，也就是令wTx+∗+b=1,wTx−∗+b=−1。此时支持向量机所要求的超平面所需要满足的条件变为(wTxi+b∥w∥⩾+1∥w∥,yi=+1wTxi+b∥w∥⩽−1∥w∥,yi=−1由于∥w∥恒大于0，因此上式可进一步化简为(wTxi+b⩾+1,yi=+1wTxi+b⩽−1,yi=−16.1.5式(6.4)的推导根据式(6.3)的推导可知，x+∗和x−∗便是“支持向量”，因此支持向量到超平面的距离已经被约束为1∥w∥，所以两个异类支持向量到超平面的距离之和为2∥w∥。6.1.6式(6.5)的解释式(6.5)是通过“最大化间隔”来保证超平面离正负样本都尽可能远，且该超平面有且仅有一个，因此可以解出唯一解。...

[来源2] 为等式，说明µ∗igi(x∗)=0。此时再结合

通过结构化输出，我们可以将模型的回答解析为程序可处理的数据结构，便于后续的自动化处理，如：
- 自动验证引用的准确性
- 将引用链接到原文
- 根据置信度决定是否需要人工审核

**参考链接与文献**

1. https://www.pinecone.io/learn/series/rag/rerankers/
2. https://towardsdatascience.com/improving-rag-performance-using-rerankers-6adda61b966d
3. https://www.llamaindex.ai/blog/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83
4. https://python.langchain.ac.cn/docs/how_to/qa_citations/
5. Zhang T, Patil S G, Jain N, et al. Raft: Adapting language model to domain specific rag[J]. arXiv preprint arXiv:2403.10131, 2024.


## 总结

本章介绍了 RAG 系统中生成阶段的关键优化技术：

### 后处理优化

**信息压缩**
- **上下文压缩**：使用 `EmbeddingsFilter` 过滤掉与查询相关性低的文档，减少 Token 消耗
- **提示压缩**：使用 LLMLingua 等工具压缩 Prompt，在保持语义完整性的同时降低成本

**重排序 (Reranking)**
- 使用 Cross-Encoder 模型（如 bge-reranker）对检索结果进行精排

### 参考引用

- 在回答中标注信息来源，提高可解释性和可信度
- 使用结构化输出（Pydantic）便于后续处理

### 最佳实践建议

1. **根据场景选择压缩策略**：简单查询用上下文压缩，复杂查询用提示压缩
2. **合理设置相似度阈值**：过高会丢失相关信息，过低则无法有效过滤
3. **重排序与压缩可组合使用**：先重排序选出 Top-K，再压缩减少 Token
4. **始终提供引用来源**：提高用户对 AI 回答的信任度